In [ ]:
import sys
sys.path.append("..")
from figutils import *
from string import ascii_lowercase

In [ ]:
output_dir = os.path.join(OUTPUT_DIR, os.path.basename(os.getcwd()))

In [ ]:
methods = ["analytic", "baseline", "recursive"]
method_names = ["Analytic", "Ours - naïve", "Ours - recursive"]
imgs = []
weights = []
it = 50
variances = []
img_ref = mi.Bitmap(os.path.join(output_dir, "img_ref.exr"))

for method in methods:
    imgs.append(mi.Bitmap(os.path.join(output_dir, method, "img", f"{it:03d}.exr")))
    weights.append(mi.Bitmap(os.path.join(output_dir, method, "weights", f"{it:03d}.exr")))
    variances.append(np.load(os.path.join(output_dir, method, "var.npy")))


In [ ]:
show_std = True
import matplotlib.patheffects as pe
n_rows = len(methods)
n_cols = 2
w,h = imgs[0].size()
wspace = 0.02
hspace = 0.02*w/h
aspect = gridspec_aspect(n_rows, n_cols, w, h, wspace, hspace)
sns.set_style('white')
fig = plt.figure(1, figsize=(TEXT_WIDTH, TEXT_WIDTH / aspect))
gs = fig.add_gridspec(n_rows, n_cols, wspace=wspace, hspace=hspace)

for i, method in enumerate(method_names):
    ax = fig.add_subplot(gs[i, 0])
    ax.imshow(mi.util.convert_to_bitmap(imgs[i]), interpolation='none')
    if i == 2:
        ax.set_title("Primal")
    # disable_border(ax)
    disable_ticks(ax)
    ax.set_ylabel(method)
    # ax.set_ylabel(f"Var: {variances[i][it]:.1e}\n" +fr"\textbf{{({ascii_lowercase[i]})}} {method}", y=-0.5)
    # ax.text(0.99*w, 0.99*h, f"Var: {variances[i][it]:.1e}", ha="right", va="bottom", color="white", fontsize=DEFAULT_FONTSIZE, path_effects=[pe.withStroke(linewidth=1, foreground="black")])
    v = variances[i][it]
    if show_std:
        v = np.sqrt(v)
    e = int(np.log10(v))
    if e == 0:
        text = f"$\sigma{'' if show_std else '^2'}={v/10**e:.3f}$"
    else:
        text = f"$\sigma{'' if show_std else '^2'}={v/10**e:.4f}\cdot 10^{{{e}}}$"
    ax.text(0.99*w, 0.99*h, text, ha="right", va="bottom", color="white", fontsize=DEFAULT_FONTSIZE, path_effects=[pe.withStroke(linewidth=1, foreground="black")])


    ax = fig.add_subplot(gs[i, 1])
    im = ax.imshow(mi.TensorXf(weights[i])[..., 0], cmap='Reds_r', vmin=0.0, vmax=1.0, interpolation='none')
    if i == 2:
        ax.set_title("Weights")
    # disable_border(ax)
    disable_ticks(ax)

    if i == 2:
        cbax = ax.inset_axes([1.02, 0, 0.04, 1], transform=ax.transAxes)
        cbar = fig.colorbar(im, cax=cbax, ticks=[0, 0.5, 1])
        cbar.outline.set_visible(False)
        cbar.ax.tick_params(size=0)


# gs.update(wspace=0.015, hspace=0.015*aspect)
save_fig('steady_state')